In [1]:
import matplotlib.pyplot as plt
import numpy as np

class DraggablePoint:
    def __init__(self, ax, x, y, color="ro", marker=6):
        """ 
        color = "ro" for 
        """
        self.ax = ax
        self.point, = ax.plot(x, y, 'ro', markersize=10)
        self.cid = self.point.figure.canvas.mpl_connect('button_press_event', self.on_click)
        self.active = False
        
    def on_click(self, event): 
        if event.inaxes == self.ax and event.button == 1:
            contains, _ = self.point.contains(event)
            if contains:
                self.active = True
                self.last_x = event.xdata
                self.last_y = event.ydata
                self.cid_motion = self.point.figure.canvas.mpl_connect('motion_notify_event', self.on_motion)
                self.cid_release = self.point.figure.canvas.mpl_connect('button_release_event', self.on_release)

    def on_motion(self, event):
        if event.inaxes == self.ax and self.active:
            dx = event.xdata - self.last_x
            dy = event.ydata - self.last_y
            self.point.set_data([event.xdata], [event.ydata])
            
            update_predictions()
            
            self.point.figure.canvas.draw()
            self.last_x = event.xdata
            self.last_y = event.ydata

    def on_release(self, event):
        if self.active:
            self.active = False
            self.point.figure.canvas.mpl_disconnect(self.cid_motion)
            self.point.figure.canvas.mpl_disconnect(self.cid_release)


def main():
    #p = Predictor(checkpoint=rf"results\{config['predictor']['basic']['checkpoint']}.pth",scaler=rf"data\scaler_X_all.pkl")
    fig, ax = plt.subplots()
    ax.set_xlim(-500, 1500)
    ax.set_ylim(-500, 1500)
    ax.set_aspect("equal")
    ax.set_title("Live predictions")

    ax.add_patch(plt.Rectangle((0, 0), 1000, 1000, ls="--", lw=2, ec="black", fc="none"))
    
    global draggable_points
    points_data = [(0, 0), (500, 0), (620, 780), (750, 340)]
    draggable_points = [DraggablePoint(ax, x, y) for x, y in points_data]

    scatter = ax.scatter([], [], c='b', marker='o', s=10) #scatter plot for predictions
    lines = ax.plot([], [], 'black')[0]
    lines.set_data([], [])  # Initialize empty line

    global update_predictions
    def update_predictions():
        x = [point.point.get_xdata()[0] for point in draggable_points]
        y = [point.point.get_ydata()[0] for point in draggable_points]
        lines.set_data(x, y)
        n4_rotz = 3
        
        coords = {"x1"   : x[0]/1000,
                  "y1"   : y[0]/1000,
                  "x2"   : x[1]/1000,
                  "y2"   : y[1]/1000,
                  "x3"   : x[2]/1000,
                  "y3"   : y[2]/1000,
                  "x4"   : x[3]/1000,
                  "y4"   : y[3]/1000,
                  "rotz" : n4_rotz}
    
        #df_results_predictions = predict_mechanism(p, coords, NSUBST=50)
        #scatter.set_offsets(np.array(df_results_predictions["coords_n1to4"].to_list())[:,1,:]*1000)
    update_predictions()
    #for point in draggable_points:
    #    point.point.figure.canvas.mpl_connect('motion_notify_event', update_lines)
    #    point.point.figure.canvas.mpl_connect('button_release_event', update_lines)
        
        

    plt.show()

if __name__ == "__main__":
    %matplotlib qt5
    main()


ImportError: Failed to import any of the following Qt binding modules: PyQt5, PySide2